# LangChain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
!pip3 install pydantic==1.10.8

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

model = ChatOpenAI()
output_parser = StrOutputParser()

In [ ]:
# Modified vectorstore to add more context
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", 
     "bears like to eat honey", 
     "Some bears have been known to eat chocolate when they find it in the wild.",
    "Harrison was born in 1985."],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [6]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='Harrison was born in 1985.'),
 Document(page_content='bears like to eat honey'),
 Document(page_content='Some bears have been known to eat chocolate when they find it in the wild.')]

In [7]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='Some bears have been known to eat chocolate when they find it in the wild.'),
 Document(page_content='harrison worked at kensho'),
 Document(page_content='Harrison was born in 1985.')]

In [8]:
retriever.get_relevant_documents("Do bears like chocolate?")

[Document(page_content='Some bears have been known to eat chocolate when they find it in the wild.'),
 Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho'),
 Document(page_content='Harrison was born in 1985.')]

In [9]:
retriever.get_relevant_documents("When was Harrison born?")

[Document(page_content='Harrison was born in 1985.'),
 Document(page_content='harrison worked at kensho'),
 Document(page_content='Some bears have been known to eat chocolate when they find it in the wild.'),
 Document(page_content='bears like to eat honey')]

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain.schema.runnable import RunnableMap

In [12]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [13]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [14]:
chain.invoke({"question": "When was Harrison born?"})

'Harrison was born in 1985.'

In [15]:
chain.invoke({"question": "Do bears like chocolate?"})

'Yes.'

In [16]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [17]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='Harrison was born in 1985.'),
  Document(page_content='bears like to eat honey'),
  Document(page_content='Some bears have been known to eat chocolate when they find it in the wild.')],
 'question': 'where did harrison work?'}